# Multi-Classification


In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import keras
import tensorflow as tf
import numpy as np
from numpy  import expand_dims
import cv2
from matplotlib import pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
np.random.seed(15)

Using TensorFlow backend.


# 1. Augmentation 했을 때

## 1.1 data Augmentation

In [12]:
batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# 검증 및 테스트 이미지는 augmentation을 적용하지 않음(이미지 원본을 사용)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# 이미지를 배치 단위로 불러와 줄 generator입니다.
train_generator = train_datagen.flow_from_directory(
        './clean-dataset/train', 
        target_size=(150, 150), 
        batch_size=batch_size,
        class_mode='categorical') 

validation_generator = validation_datagen.flow_from_directory(
        './clean-dataset/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        'clean-dataset/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

Found 600 images belonging to 3 classes.
Found 150 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


## 1. 2. model

In [6]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

## 1.3. train

In [7]:
# steps_per_epoch는 한 세대마다 몇 번 생성기로부터 데이터를 얻을지를 나타내는 값
# 한 세대마다 사용되는 학습데이터의 수는 steps_per_epoch * batch_size
        
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,    # 2000/16     한번에 125개씩 생성
        epochs=5,  #50
        validation_data=validation_generator,
        validation_steps=800 // batch_size)     # 800/16   한번에 50개씩 생성

Epoch 1/5
125/125 [==============================] - 30s 243ms/step - loss: 0.6986 - accuracy: 0.6974 - val_loss: 0.3453 - val_accuracy: 0.6133
Epoch 2/5
125/125 [==============================] - 29s 229ms/step - loss: 0.4762 - accuracy: 0.8303 - val_loss: 0.4157 - val_accuracy: 0.8600
Epoch 3/5
125/125 [==============================] - 29s 232ms/step - loss: 0.4051 - accuracy: 0.8548 - val_loss: 0.0291 - val_accuracy: 0.8400
Epoch 4/5
125/125 [==============================] - 29s 235ms/step - loss: 0.3582 - accuracy: 0.8684 - val_loss: 0.2404 - val_accuracy: 0.8200
Epoch 5/5
125/125 [==============================] - 29s 232ms/step - loss: 0.3013 - accuracy: 0.8810 - val_loss: 0.0530 - val_accuracy: 0.8933


## 1. 4. evaluate

In [13]:
# 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator( test_generator, steps = 5)
print(scores[1])

-- Evaluate --
0.887499988079071
